# Model Controller Tutorial: Classification

> This notebook contains an end-to-end process of preprocess + tokenizing your text, and build a classification model based on Roberta architecture

- skip_showdoc: true
- skip_exec: true

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
from importlib.machinery import SourceFileLoader
import os
from transformers import DataCollatorWithPadding,RobertaTokenizer
from transformers.models.roberta.modeling_roberta import RobertaModel,RobertaForSequenceClassification
import torch
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
import pandas as pd
import numpy as np

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
pd.Series(list(map(lambda x: len(x.split()),[text for text in dset['Review Text'] if text is not None]))).describe()

count    22641.000000
mean        60.196679
std         28.534612
min          2.000000
25%         36.000000
50%         59.000000
75%         88.000000
max        115.000000
dtype: float64

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title','Division Name'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         # add "str.lower" here because nearby_aug might return uppercase character
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=True
                        )

Define our tokenizer for Roberta

In [ ]:
_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

TODO: do the following experiments

- set max_length smaller (100), benchmark training time. Then benchmark validation prediction (hoping no error)
- dont set max_length, set data_collator, benchmark validation prediction

Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
----- lower -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Text Augmentation --------------------
----- nlp_aug_stochastic -----


Map (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

----- lower -----


Map (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

Let's see one example of how those content transformations and augmentations affect our input

In [ ]:
sample_txt = 'This is not what I expected 🤬. I gulped when I put this in my bag during retailer days because the price was still too much ... but thought this has to be wonderful to charge so much,right??'
print(sample_txt)

This is not what I expected 🤬. I gulped when I put this in my bag during retailer days because the price was still too much ... but thought this has to be wonderful to charge so much,right??


In [ ]:
two_steps_tokenization_explain(sample_txt,_tokenizer,
                               content_tfms=[text_normalize,str.lower],
                               aug_tfms=[partial(nlp_aug_stochastic,aug=aug,p=1),str.lower]
                              )

		------- Text Transformation Explained -------
----- Raw sentence -----
This is not what I expected 🤬. I gulped when I put this in my bag during retailer days because the price was still too much ... but thought this has to be wonderful to charge so much,right??

----- Content Transformations (on both train and test) -----
--- text_normalize ---
This is not what I expected 🤬 . I gulped when I put this in my bag during retailer days because the price was still too much ... but thought this has to be wonderful to charge so much , right ? ?

--- lower ---
this is not what i expected 🤬 . i gulped when i put this in my bag during retailer days because the price was still too much ... but thought this has to be wonderful to charge so much , right ? ?


----- Augmentations (on train only) -----
--- nlp_aug_stochastic ---
tMis is not what i expected 🤬. i gulped when i put this in my bag during rrtailer Cays because the price was still too much. .. but thought this has to be wonderful to Vharg

# Model Experiment: Roberta Vanilla Single-Head Classification

In [ ]:
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


## Train Roberta model using the Model Controller

In [ ]:
tdc.label_lists[0]

['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']

In [ ]:
num_classes = len(tdc.label_lists[0])
num_classes

6

Let's define our model 

In [ ]:
model_name='roberta-base'

In [ ]:
_model = RobertaForSequenceClassification.from_pretrained(model_name,num_labels=num_classes)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
_model = _model.to('cuda:0')

Then we can define the metrics to used, and the Model Controller object

In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score] 
# we will use both f1_macro and accuracy score as metrics

In [ ]:
controller = ModelController(_model,
                             data_store=tdc,
                             metric_funcs=metric_funcs)

And we can start training our model

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 2

controller.fit(epochs,lr,
               head_sizes=num_classes,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score Department name,Accuracy Score Department name
1,No log,0.279182,0.749650,0.918471
2,0.395900,0.259449,0.752880,0.923553


### Logging your training

You can log your training using HuggingFace:

- Supported platforms are "azure_ml", "comet_ml", "mlflow", "neptune", "tensorboard","clearml" and "wandb"

- References:

    - https://huggingface.co/docs/transformers/v4.28.0/en/main_classes/trainer#transformers.TrainingArguments
    
    - https://docs.wandb.ai/guides/integrations/huggingface#:~:text=Logging%20your%20Hugging%20Face%20model,every%20save_steps%20in%20the%20TrainingArguments%20.

```python
controller.fit(epochs,lr,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
               hf_report_to='wandb'
              )
```

You can save your model weights at the end of your training

```python
controller.trainer.model.save_pretrained('./sample_weights/model_progress')
```

Or you can save your weights at every epochs during your training

```python
controller.fit(epochs,lr,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=True,
               o_dir='sample_weights',
               compute_metrics=compute_metrics_classification,
              )
```

## Train model with tokenized DatasetDict (no TextDataController)

This part assumes you already have your tokenized datasetdict, without the use of `TextDataController`

In [ ]:
import copy

In [ ]:
main_ddict = copy.deepcopy(tdc.main_ddict)

In [ ]:
main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18102
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

Note that your DatasetDict must contain tokens besides raw text (which typically includes 'input_ids', 'token_type_ids', 'attention_mask')

In [ ]:
num_classes = 6

In [ ]:
model_name='roberta-base'

_model = RobertaForSequenceClassification.from_pretrained(model_name,num_labels=num_classes)

_model = _model.to('cuda:0')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score] 

# note that you omit the `data_store` argument
controller = ModelController(_model,metric_funcs=metric_funcs)

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 2

controller.fit(epochs,lr,
               ddict=main_ddict, # Put in your tokenized datasetdict here
               label_names='Department Name',
               head_sizes=num_classes,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
               tokenizer=_tokenizer,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score Department name,Accuracy Score Department name
1,No log,0.281601,0.748485,0.917145
2,0.383900,0.263439,0.750939,0.922669


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/model_progress')

## Predict using trained model, using TDM

### Load trained model

In [ ]:
_model = RobertaForSequenceClassification.from_pretrained('./sample_weights/model_progress',num_labels=6)

_model = _model.to('cuda:0')

In [ ]:
_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(_model,tdc,metric_funcs)

### Predict Train/Validation set

Make prediction on all validation set

In [ ]:
len(_tmp.split('\n'))

906

In [ ]:
for i in tdc.main_ddict['train']['input_ids']:
    print(i.shape)

torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size([162])
torch.Size

In [ ]:
for i in tdc.main_ddict['validation']['input_ids']:
    print(i.shape)

torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size([144])
torch.Size

In [ ]:
df_val = controller.predict_ddict_classification(ds_type='validation')

[[0, 15841, 4716, 1459, 479, 479, 215, 10, 1531, 8443, 27785, 372, 7, 3568, 11, 5, 2428, 50, 7, 5, 558, 25, 41, 3626, 7, 10, 4828, 3089, 14609, 479, 182, 3473, 27785, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 15841, 4716, 1459, 479, 2007, 8, 14878, 479, 939, 802, 42, 6399, 21, 269, 27360, 8, 14878, 479, 129, 12071, 4376, 16, 103, 9, 5, 15533, 66, 6538, 56, 10199, 202, 7391, 61, 47, 33, 847, 160, 19, 36063, 111, 3680, 5, 6399, 40, 29650, 479, 8, 5, 200, 696, 9, 2671, 3585, 32, 5, 614, 3124, 31670, 479, 3739, 9, 11689, 2018, 111, 45, 269, 686, 141, 7, 120, 198, 14, 98, 939, 460, 3568, 24, 19, 10, 1886, 6703, 479, 53, 24, 74, 28, 2579, 45, 7, 33, 7, 479, 97, 87, 14, 24, 1326, 2579, 8, 15029, 15481,

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

{'Title': ['', 'simple and elegant', 'retro and pretty', 'summer/fall wear', 'perfect except slip', '', 'lovely socks', 'perfect for petites -- please keep making these!', '', 'not for me', "cute... if you're planning to hold it on", 'super soft', 'great for hot summers', 'bulky, unflattering', 'lovely for a little while', 'too boxy but nice idea'], 'Review Text': ['general petite . . such a fun jacket ! great to wear in the spring or to the office as an alternative to a formal blazer . very comfortable !', 'general petite . simple and elegant . i thought this shirt was really feminine and elegant . only downsides is some of the punched out holes had fabric still attached which you have cut off with scissors - otherwise the shirt will snag . and the second issue of bigger importance are the low armholes . lots of bra showing - not really sure how to get around that so i always wear it with a cardigan . but it would be nice not to have to . other than that it looks nice and pairs nicely

{'Title': ['love the stripes', '', 'comfortable cool', '', 'gorgeous', 'beautiful details, but not quite right', 'effortlessly cool', 'super cute & comfy', 'too big and baggy', 'neck', 'effortless, easy summer staple', 'pretty tunic', 'sadly nothing as shown on model', 'pretty in peplum', 'perfect for summer evenings', 'little black winter dress'], 'Review Text': ["general . love the stripes . i tried both the xs and s in the store . i am usually an xs , but ended up buying the s , because the xs was really short and i didn't want it to be any shorter after washing . the s fit more like a normal t-shirt , although the back is longer . love the color ( blue ) and the fabric is super soft . also happy that it's not too thin , nor see-through .", 'general . . this fits like a dream ! the almost three quarter length sleeves are perfection . worth every penny !', 'general petite . comfortable cool . easy to wear , but still has a lot of style . i see this becoming a new basic .', "general .

{'Title': ['clingy material', 'nice top, but runs small', 'pretty tank', 'boho chic', 'so amazing', 'beautiful', 'simple but overpriced', 'not cut like a traditional button down', 'stunning - but not for the faint of heart.', 'try on before purchasing', 'perfect jumper', 'love!', 'sleeves stretch out', 'neck tight', 'a little boxy', 'cute trendy top'], 'Review Text': ['general petite . clingy material . the shirt looks a lot less fitted on the model in the picture than how it fit in " real " life . size was not wrong , the material was just very clingy . i was not impressed and subsequently returned it .', "general . nice top , but runs small . i like the fabric and aesthetic of this top , and really wanted to like it , however i wish it were longer . i am a size small or med in all retailer clothes , but even the medium felt short and clingy . however that might be because i'm curvy with a long torso , so i'd definitely recommend trying this on rather than buying online !", 'general p

AttributeError: 'list' object has no attribute 'to'

In [ ]:
%debug

> /home/quan/kwon/that-nlp-library/that_nlp_library/model_main.py(277)<dictcomp>()
    275         batch = data_collator(collator_inp)
    276 
--> 277     inputs = {k:v.to(device) for k,v in batch.items()
    278               if k in model_input_names}
    279 

ipdb> list
    272         for pair in zip(*vs):
    273             collator_inp.append({k:v for k,v in zip(ks,pair)})
    274 
    275         batch = data_collator(collator_inp)
    276 
--> 277     inputs = {k:v.to(device) for k,v in batch.items()
    278               if k in model_input_names}
    279 
    280     _f = partial(torch.nn.functional.softmax,dim=1) if not is_multilabel else torch.sigmoid
    281 
    282     # switch to eval mode for evaluation

ipdb> model_input_names
['input_ids', 'attention_mask']
ipdb> print(k)
input_ids
ipdb> batch.keys()
*** NameError: name 'batch' is not defined
ipdb> print(type(v))
<class 'list'>
ipdb> print(v[:2])
[tensor([    0, 15841,  4716,  1459,   479,    42,    16,    10,   3

In [ ]:
tdc.main_ddict['train']['input_ids'][:2]

[tensor([    0, 15841,  4716,  1459,     4,   162,   298,     4,  3553,  6634,
         27115,   636,    16,   169,    81,  7663,    13,     5,   992,  2553,
           459,     8,  2677,  3853,  1571,     4,    24,  2564, 15995,    36,
         39478,    10,  1836,  2514,    43,    53,    24,   128,   579,    45,
           269, 34203,     6,    24,  1236,  1182,   449,  2832,     9, 23835,
            89,   546, 15983,     4,    24,    16,    10,   410,   350,  1844,
             9,    10,   748,   847,    13,    10,   173,   299,    25,   157,
             4,    42,   299,   109, 13184,    45,   323,     5,   425,    23,
            70,     4,    24,  1299,   101, 45420, 11040,   298,   939,   115,
           465,    23,  1494,  1400,    13,   169,   540,     4,   939,    40,
            28,  5494,   257,  4226,    24,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [ ]:
df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,3,google play,Commercial,0.837496
1,google play - Zq,5,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,5,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,6,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,2,google play,Delivery,0.758327


To convert the label index to string, we can use the ```label_lists``` attribute of tdm

In [ ]:
df_val['label']= df_val['label'].apply(lambda x: tdm.label_lists[0][x]).values

df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,Feature,google play,Commercial,0.837496
1,google play - Zq,Others,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,Others,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,Payment,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,Delivery,google play,Delivery,0.758327


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val.label,df_val.pred_L1,average='macro')

0.4634417008698494

You can also make predictions on all training set, by changing argument ```ds_type``` to "train"

### Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = TextDataMain.from_csv(Path('sample_data')/'sample_large.csv',return_df=True)

----- Input Validation Precheck -----
DataFrame contains duplicated values!
-----> Number of duplications: 16 rows


We will remove all the labels and unnecessary columns

In [ ]:
true_label = df_test['L1'].values

In [ ]:
df_test = df_test.drop(['L1','L2'],axis=1)

In [ ]:
df_test.head()

,Source,Content
0,Google Play,"App ncc lúc nào cx lag đơ, phần tìm kiếm thì v..."
1,Non Owned,..❗️ GÓC THANH LÝ Tính ra rẻ hơn cả mua #Shope...
2,Google Play,Mắc gì người ta đặt hàng toàn lỗi 😃????
3,Owned,#GhienShopeePayawardT8 Khi bạn chơi shopee quá...
4,Google Play,Rất bức xúc khi dùng . mã giảm giá người dùng ...


We will create a DatasetDict for this test dataframe

In [ ]:
test_ddict = tdm.get_test_datasetdict_from_df(df_test)

-------------------- Getting Test Set --------------------
----- Input Validation Precheck -----
DataFrame contains duplicated values!
-----> Number of duplications: 19 rows
-------------------- Start Test Set Transformation --------------------
----- Metadata Simple Processing & Concatenating to Main Content -----
-------------------- Text Transformation --------------------
----- text_normalize -----


100%|█████████████████████████████████████| 2269/2269 [00:00<00:00, 3954.30it/s]


-------------------- Test Leak Checking --------------------
- Before leak check
Size: 2269
- After leak check
Size: 0
- Number of rows leaked: 2269, or 100.00% of the original validation (or test) data
-------------------- Construct DatasetDict --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

Remember the ***Leak Check*** we did in TextDataMain? Our ```df_test``` only has 70 rows, and it also shows that 70 rows of our data is leaked (100%), which is correct because this test dataset is actually a small sample of the training data.

In [ ]:
test_ddict

DatasetDict({
    test: Dataset({
        features: ['text', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2269
    })
})

Our test data has been processed + transformed (but not augmented) the same way as the validation set. Now we can start making the prediction

In [ ]:
controller = ModelController(trained_model,tdm)
df_result = controller.predict_ddict(ddict=test_ddict,ds_type='test')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

In [ ]:
df_result.head()

,text,Source,pred_L1,pred_prob_L1
0,"google play - App ncc lúc nào cx lag đơ , phần...",google play,Feature,0.878221
1,non owned - .. ❗ ️ GÓC THANH LÝ Tính ra rẻ hơn...,non owned,Others,0.930981
2,google play - Mắc gì người ta đặt hàng toàn lỗ...,google play,Feature,0.849374
3,owned - # GhienShopeePayawardT8 Khi bạn chơi s...,owned,Commercial,0.915552
4,google play - Rất bức xúc khi dùng . mã giảm g...,google play,Shopee account,0.571471


Let's quickly check the f1 score to make sure everything works correctly

In [ ]:
f1_score(true_label,df_result.pred_L1,average='macro')

0.5303012712104336

Since we are getting the predictions on the entire training+validation set, the F1 score is expected to be slightly higher than validation's F1 score.

We can even predict top k results

In [ ]:
df_result = controller.predict_ddict(ddict=test_ddict,ds_type='test',topk=3)
df_result.head()

-------------------- Start making predictions --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

,text,Source,pred_L1,pred_prob_L1,pred_L1_top1,pred_L1_top2,pred_L1_top3,pred_prob_L1_top1,pred_prob_L1_top2,pred_prob_L1_top3
0,"google play - App ncc lúc nào cx lag đơ , phần...",google play,"[3, 5, 9]","[0.87822074, 0.023822138, 0.02159522]",Feature,Others,Shopee account,0.878221,0.023822,0.021595
1,non owned - .. ❗ ️ GÓC THANH LÝ Tính ra rẻ hơn...,non owned,"[5, 1, 0]","[0.9309808, 0.015578598, 0.009805982]",Others,Commercial,Buyer complained seller,0.930981,0.015579,0.009806
2,google play - Mắc gì người ta đặt hàng toàn lỗ...,google play,"[3, 5, 9]","[0.8493735, 0.050054528, 0.021759989]",Feature,Others,Shopee account,0.849374,0.050055,0.021760
3,owned - # GhienShopeePayawardT8 Khi bạn chơi s...,owned,"[1, 6, 7]","[0.9155516, 0.01255093, 0.010521941]",Commercial,Payment,Return/Refund,0.915552,0.012551,0.010522
4,google play - Rất bức xúc khi dùng . mã giảm g...,google play,"[9, 3, 1]","[0.57147133, 0.25687057, 0.03061041]",Shopee account,Feature,Commercial,0.571471,0.256871,0.030610


If we just want to make a prediction on a small amount of data (single sentence, or a few sentences), we can use `ModelController.predict_raw_text`

In [ ]:
# Since we have some metadatas, we need to define a dictionary (to imitate a DatasetDict)
raw_content={
    'Source': 'Google play',
    'Content':'Tôi không thích Shopee.Tại vì dùng app rất chậm,lag banh nhà lầu, thậm chí log in còn không đc'
}

If we don't use metadata, we can use something like this: 

```raw_content='Tôi không thích Shopee.Tại vì dùng app rất chậm,lag banh nhà lầu, thậm chí log in còn không đc'```

In [ ]:
df_result = controller.predict_raw_text(raw_content,topk=1)
df_result

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4718.00it/s]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

,text,Source,pred_L1,pred_prob_L1
0,google play - Tôi không thích Shopee . Tại vì ...,google play,Feature,0.876843


## Predict using trained model, using tokenized DatasetDict

### Load trained model

In [ ]:
num_classes = 10

model_name='nguyenvulebinh/envibert'
_model_kwargs={
    # overall model hyperparams
    'is_multilabel':False, # False
    'is_multihead':False, # False
    'head_class_sizes':num_classes,
    'head_class': SimpleClassificationHead,
    # classfication head hyperparams
    'classifier_dropout':0.1 
}

In [ ]:
trained_model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/model_progress'), 
                                          output_hidden_states=False,
                                          seed=42,
                                          model_kwargs = _model_kwargs)


metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(trained_model,metric_funcs) # notice that we don't use tdm here

Some weights of the model checkpoint at sample_weights/model_progress were not used when initializing RobertaBaseForSequenceClassification: ['body_model.pooler.dense.weight', 'body_model.pooler.dense.bias']
- This IS expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Predict validation set

In [ ]:
main_ddict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4780
    })
    validation: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask', 'pred_L1', 'pred_prob_L1'],
        num_rows: 447
    })
})

In [ ]:
my_label_name = 'L1'
my_class_predefined = ['Buyer complained seller',
 'Commercial',
 'Delivery',
 'Feature',
 'Order/Item',
 'Others',
 'Payment',
 'Return/Refund',
 'Services',
 'Shopee account']

In [ ]:
df_val = controller.predict_ddict(main_ddict,
                                  ds_type='validation',
                                  is_multilabel=False,
                                  tokenizer=tokenizer,
                                  label_names = my_label_name,
                                  class_names_predefined=my_class_predefined
                                  )

-------------------- Start making predictions --------------------


Map:   0%|          | 0/447 [00:00<?, ? examples/s]

In [ ]:
df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,3,google play,Commercial,0.837496
1,google play - Zq,5,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,5,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,6,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,2,google play,Delivery,0.758327


### Predict test set

In [ ]:
test_ddict

DatasetDict({
    test: Dataset({
        features: ['text', 'Source', 'input_ids', 'token_type_ids', 'attention_mask', 'pred_L1', 'pred_prob_L1'],
        num_rows: 2269
    })
})

It would be cumbersome to preprocess your test data the same way you preprocess your validation set, without the use of tdm (which stores the preprocess pipeline). In short, you need to produce the test datasetdict `test_ddict` containing processed `'input_ids', 'token_type_ids', 'attention_mask'`, then call

```python
df_results = controller.predict_ddict(ddict=test_ddict,
                                      ds_type='test',
                                      is_multilabel=False,
                                      tokenizer=tokenizer,
                                      label_names = my_label_name,
                                      class_names_predefined=my_class_predefined     
                                     )
```

# Model Experiment: Roberta Custom Single-Head Classification

In [ ]:
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Train EnviBert using TDM

In [ ]:
tdc.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
num_classes = len(tdc.label_lists[0])
num_classes

6

Let's define our model and model controller. First, we will initialize the pretrained `body` model

In [ ]:
from transformers.models.roberta.modeling_roberta import RobertaModel

In [ ]:
model_name='nguyenvulebinh/envibert'

In [ ]:
envibert_body = RobertaModel.from_pretrained(model_name)

Some weights of the model checkpoint at nguyenvulebinh/envibert were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Then we can define a simple class as the head for our classification task, something like this:

In [ ]:
class SimpleClassificationHead(torch.nn.Module):
    def __init__(self,
                 config, # HuggingFace model configuration
                 classifier_dropout=0.1, # Dropout ratio (for dropout layer right before the last nn.Linear)
                 num_labels=None, # Number of label output. Every classification class must have this exact variable
                ):
        super().__init__()
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.out_proj = torch.nn.Linear(config.hidden_size, num_labels)
    def forward(self, inp, **kwargs):
        x = inp
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
_model_kwargs={
    # overall model hyperparams
    'is_multilabel':tdm.is_multilabel, # False
    'is_multihead':tdm.is_multihead, # False
    'head_class_sizes':num_classes,
    'head_class': SimpleClassificationHead,
    # classfication head hyperparams
    'classifier_dropout':0.1 
}

model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                  cpoint_path = 'nguyenvulebinh/envibert', 
                                  output_hidden_states=False, # since we are not using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=envibert_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score] # we will use both f1_macro and accuracy score as metrics
controller = ModelController(model,tdm,metric_funcs)

Loading body weights. This assumes the body is the very first first-layer block of your custom architecture


And we can start training our model

In [ ]:
lr = 8e-5
bs=4
wd=0.01
epochs= 2

controller.fit(epochs,lr,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L1,Accuracy Score L1
0,No log,1.059704,0.350748,0.677852
1,No log,1.007712,0.462641,0.697987


### Logging your training

You can log your training using HuggingFace:

- Supported platforms are "azure_ml", "comet_ml", "mlflow", "neptune", "tensorboard","clearml" and "wandb"

- References:

    - https://huggingface.co/docs/transformers/v4.28.0/en/main_classes/trainer#transformers.TrainingArguments
    
    - https://docs.wandb.ai/guides/integrations/huggingface#:~:text=Logging%20your%20Hugging%20Face%20model,every%20save_steps%20in%20the%20TrainingArguments%20.

```python
controller.fit(epochs,lr,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
               hf_report_to='wandb'
              )
```

You can save your model weights at the end of your training

```python
controller.trainer.model.save_pretrained('./sample_weights/model_progress')
```

Or you can save your weights at every epochs during your training

```python
controller.fit(epochs,lr,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=True,
               o_dir='sample_weights',
               compute_metrics=compute_metrics_classification,
              )
```

## Train EnviBert with tokenized DatasetDict

This part assumes you already have your tokenized datasetdict. You must have your tokenizer as well

In [ ]:
tokenizer

RobertaTokenizer(name_or_path='', vocab_size=59993, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True)

Note that your DatasetDict must contain tokens besides raw text (which typically includes 'input_ids', 'token_type_ids', 'attention_mask')

In [ ]:
main_ddict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4780
    })
    validation: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 447
    })
})

In [ ]:
num_classes = 10

In [ ]:
model_name='nguyenvulebinh/envibert'
_model_kwargs={
    # overall model hyperparams
    'is_multilabel':False, # False
    'is_multihead':False, # False
    'head_class_sizes':num_classes,
    'head_class': SimpleClassificationHead,
    # classfication head hyperparams
    'classifier_dropout':0.1 
}

In [ ]:
envibert_body = RobertaModel.from_pretrained(model_name)

In [ ]:
model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                  cpoint_path = 'nguyenvulebinh/envibert', 
                                  output_hidden_states=False, # since we are not using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=envibert_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score] # we will use both f1_macro and accuracy score as metrics
controller = ModelController(model,
                             metric_funcs=metric_funcs)

Some weights of the model checkpoint at nguyenvulebinh/envibert were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading body weights. This assumes the body is the very first first-layer block of your custom architecture


In [ ]:
lr = 8e-5
bs=4
wd=0.01
epochs= 2

controller.fit(epochs,lr,
               ddict=main_ddict, # Put in your tokenized datasetdict here
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
               tokenizer=tokenizer,
               label_names='L1'
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L1,Accuracy Score L1
0,No log,1.059704,0.350748,0.677852
1,No log,1.007712,0.462641,0.697987


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/model_progress')

## Predict using trained model, using TDM

### Load trained model

In [ ]:
_model_kwargs

{'is_multilabel': False,
 'is_multihead': False,
 'head_class_sizes': 10,
 'head_class': __main__.SimpleClassificationHead,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/model_progress'), 
                                          output_hidden_states=False,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

Some weights of the model checkpoint at sample_weights/model_progress were not used when initializing RobertaBaseForSequenceClassification: ['body_model.pooler.dense.weight', 'body_model.pooler.dense.bias']
- This IS expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trained_model

RobertaBaseForSequenceClassification(
  (body_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(59993, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
        

In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score] # we will use both f1_macro and accuracy score as metrics
controller = ModelController(trained_model,tdm,metric_funcs)

### Predict Train/Validation set

Make prediction on all validation set

In [ ]:
df_val = controller.predict_ddict(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/447 [00:00<?, ? examples/s]

In [ ]:
df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,3,google play,Commercial,0.837496
1,google play - Zq,5,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,5,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,6,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,2,google play,Delivery,0.758327


To convert the label index to string, we can use the ```label_lists``` attribute of tdm

In [ ]:
df_val['label']= df_val['label'].apply(lambda x: tdm.label_lists[0][x]).values

df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,Feature,google play,Commercial,0.837496
1,google play - Zq,Others,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,Others,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,Payment,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,Delivery,google play,Delivery,0.758327


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val.label,df_val.pred_L1,average='macro')

0.4634417008698494

You can also make predictions on all training set, by changing argument ```ds_type``` to "train"

### Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = TextDataMain.from_csv(Path('sample_data')/'sample_large.csv',return_df=True)

----- Input Validation Precheck -----
DataFrame contains duplicated values!
-----> Number of duplications: 16 rows


We will remove all the labels and unnecessary columns

In [ ]:
true_label = df_test['L1'].values

In [ ]:
df_test = df_test.drop(['L1','L2'],axis=1)

In [ ]:
df_test.head()

,Source,Content
0,Google Play,"App ncc lúc nào cx lag đơ, phần tìm kiếm thì v..."
1,Non Owned,..❗️ GÓC THANH LÝ Tính ra rẻ hơn cả mua #Shope...
2,Google Play,Mắc gì người ta đặt hàng toàn lỗi 😃????
3,Owned,#GhienShopeePayawardT8 Khi bạn chơi shopee quá...
4,Google Play,Rất bức xúc khi dùng . mã giảm giá người dùng ...


We will create a DatasetDict for this test dataframe

In [ ]:
test_ddict = tdm.get_test_datasetdict_from_df(df_test)

-------------------- Getting Test Set --------------------
----- Input Validation Precheck -----
DataFrame contains duplicated values!
-----> Number of duplications: 19 rows
-------------------- Start Test Set Transformation --------------------
----- Metadata Simple Processing & Concatenating to Main Content -----
-------------------- Text Transformation --------------------
----- text_normalize -----


100%|█████████████████████████████████████| 2269/2269 [00:00<00:00, 3954.30it/s]


-------------------- Test Leak Checking --------------------
- Before leak check
Size: 2269
- After leak check
Size: 0
- Number of rows leaked: 2269, or 100.00% of the original validation (or test) data
-------------------- Construct DatasetDict --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

Remember the ***Leak Check*** we did in TextDataMain? Our ```df_test``` only has 70 rows, and it also shows that 70 rows of our data is leaked (100%), which is correct because this test dataset is actually a small sample of the training data.

In [ ]:
test_ddict

DatasetDict({
    test: Dataset({
        features: ['text', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2269
    })
})

Our test data has been processed + transformed (but not augmented) the same way as the validation set. Now we can start making the prediction

In [ ]:
controller = ModelController(trained_model,tdm)
df_result = controller.predict_ddict(ddict=test_ddict,ds_type='test')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

In [ ]:
df_result.head()

,text,Source,pred_L1,pred_prob_L1
0,"google play - App ncc lúc nào cx lag đơ , phần...",google play,Feature,0.878221
1,non owned - .. ❗ ️ GÓC THANH LÝ Tính ra rẻ hơn...,non owned,Others,0.930981
2,google play - Mắc gì người ta đặt hàng toàn lỗ...,google play,Feature,0.849374
3,owned - # GhienShopeePayawardT8 Khi bạn chơi s...,owned,Commercial,0.915552
4,google play - Rất bức xúc khi dùng . mã giảm g...,google play,Shopee account,0.571471


Let's quickly check the f1 score to make sure everything works correctly

In [ ]:
f1_score(true_label,df_result.pred_L1,average='macro')

0.5303012712104336

Since we are getting the predictions on the entire training+validation set, the F1 score is expected to be slightly higher than validation's F1 score.

We can even predict top k results

In [ ]:
df_result = controller.predict_ddict(ddict=test_ddict,ds_type='test',topk=3)
df_result.head()

-------------------- Start making predictions --------------------


Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

,text,Source,pred_L1,pred_prob_L1,pred_L1_top1,pred_L1_top2,pred_L1_top3,pred_prob_L1_top1,pred_prob_L1_top2,pred_prob_L1_top3
0,"google play - App ncc lúc nào cx lag đơ , phần...",google play,"[3, 5, 9]","[0.87822074, 0.023822138, 0.02159522]",Feature,Others,Shopee account,0.878221,0.023822,0.021595
1,non owned - .. ❗ ️ GÓC THANH LÝ Tính ra rẻ hơn...,non owned,"[5, 1, 0]","[0.9309808, 0.015578598, 0.009805982]",Others,Commercial,Buyer complained seller,0.930981,0.015579,0.009806
2,google play - Mắc gì người ta đặt hàng toàn lỗ...,google play,"[3, 5, 9]","[0.8493735, 0.050054528, 0.021759989]",Feature,Others,Shopee account,0.849374,0.050055,0.021760
3,owned - # GhienShopeePayawardT8 Khi bạn chơi s...,owned,"[1, 6, 7]","[0.9155516, 0.01255093, 0.010521941]",Commercial,Payment,Return/Refund,0.915552,0.012551,0.010522
4,google play - Rất bức xúc khi dùng . mã giảm g...,google play,"[9, 3, 1]","[0.57147133, 0.25687057, 0.03061041]",Shopee account,Feature,Commercial,0.571471,0.256871,0.030610


If we just want to make a prediction on a small amount of data (single sentence, or a few sentences), we can use `ModelController.predict_raw_text`

In [ ]:
# Since we have some metadatas, we need to define a dictionary (to imitate a DatasetDict)
raw_content={
    'Source': 'Google play',
    'Content':'Tôi không thích Shopee.Tại vì dùng app rất chậm,lag banh nhà lầu, thậm chí log in còn không đc'
}

If we don't use metadata, we can use something like this: 

```raw_content='Tôi không thích Shopee.Tại vì dùng app rất chậm,lag banh nhà lầu, thậm chí log in còn không đc'```

In [ ]:
df_result = controller.predict_raw_text(raw_content,topk=1)
df_result

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4718.00it/s]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

,text,Source,pred_L1,pred_prob_L1
0,google play - Tôi không thích Shopee . Tại vì ...,google play,Feature,0.876843


## Predict using trained model, using tokenized DatasetDict

### Load trained model

In [ ]:
num_classes = 10

model_name='nguyenvulebinh/envibert'
_model_kwargs={
    # overall model hyperparams
    'is_multilabel':False, # False
    'is_multihead':False, # False
    'head_class_sizes':num_classes,
    'head_class': SimpleClassificationHead,
    # classfication head hyperparams
    'classifier_dropout':0.1 
}

In [ ]:
trained_model = model_init_classification(model_class = RobertaBaseForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/model_progress'), 
                                          output_hidden_states=False,
                                          seed=42,
                                          model_kwargs = _model_kwargs)


metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(trained_model,metric_funcs) # notice that we don't use tdm here

Some weights of the model checkpoint at sample_weights/model_progress were not used when initializing RobertaBaseForSequenceClassification: ['body_model.pooler.dense.weight', 'body_model.pooler.dense.bias']
- This IS expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaBaseForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Predict validation set

In [ ]:
main_ddict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4780
    })
    validation: Dataset({
        features: ['text', 'label', 'Source', 'input_ids', 'token_type_ids', 'attention_mask', 'pred_L1', 'pred_prob_L1'],
        num_rows: 447
    })
})

In [ ]:
my_label_name = 'L1'
my_class_predefined = ['Buyer complained seller',
 'Commercial',
 'Delivery',
 'Feature',
 'Order/Item',
 'Others',
 'Payment',
 'Return/Refund',
 'Services',
 'Shopee account']

In [ ]:
df_val = controller.predict_ddict(main_ddict,
                                  ds_type='validation',
                                  is_multilabel=False,
                                  tokenizer=tokenizer,
                                  label_names = my_label_name,
                                  class_names_predefined=my_class_predefined
                                  )

-------------------- Start making predictions --------------------


Map:   0%|          | 0/447 [00:00<?, ? examples/s]

In [ ]:
df_val.head()

,text,label,Source,pred_L1,pred_prob_L1
0,google play - Chơi gam rất là lác,3,google play,Commercial,0.837496
1,google play - Zq,5,google play,Others,0.927602
2,non owned - Làn sóng kỹ thuật số và sự lựa chọ...,5,non owned,Others,0.918241
3,google play - Hàng quốc tế không còn ship COD ...,6,google play,Delivery,0.804539
4,google play - Quá tệ . Giao hàng chậm như rùa ...,2,google play,Delivery,0.758327


### Predict test set

In [ ]:
test_ddict

DatasetDict({
    test: Dataset({
        features: ['text', 'Source', 'input_ids', 'token_type_ids', 'attention_mask', 'pred_L1', 'pred_prob_L1'],
        num_rows: 2269
    })
})

It would be cumbersome to preprocess your test data the same way you preprocess your validation set, without the use of tdm (which stores the preprocess pipeline). In short, you need to produce the test datasetdict `test_ddict` containing processed `'input_ids', 'token_type_ids', 'attention_mask'`, then call

```python
df_results = controller.predict_ddict(ddict=test_ddict,
                                      ds_type='test',
                                      is_multilabel=False,
                                      tokenizer=tokenizer,
                                      label_names = my_label_name,
                                      class_names_predefined=my_class_predefined     
                                     )
```